<a href="https://colab.research.google.com/github/SvetlanaErmakova/Analytics-and-statistics/blob/main/Cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого выясним, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные.

1.	Загрузим данные, проверим правильность, наличие пропущенных значений, типы данных.
2.	Создадим новый признак – марку автомобиля (company). Далее исправим названия и проверим изменения.
3.	Оставим только часть предикторов, после чего посчитаем корреляцию между price и другими переменными.
4.	Преобразуем категориальные переменные с помощью pd.get_dummies().
5.	Построим модель с одним предиктором цены – horsepower.
6.	Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Выберем какую модель лучше оставить?
7.	Попробуем избавиться от мультиколлинеарности



In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc = {'figure.figsize': (12, 6)}, style = "whitegrid")

In [ ]:
cars = pd.read_csv("https://stepik.org/media/attachments/lesson/387691/cars.csv")

##1. Загрузим данные, проверим правильность, наличие пропущенных значений, типы данных.

In [ ]:
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [ ]:
cars.dtypes

,0
car_ID,int64
symboling,int64
CarName,object
fueltype,object
aspiration,object
doornumber,object
carbody,object
drivewheel,object
enginelocation,object
wheelbase,float64


In [ ]:
cars.isna().sum()

,0
car_ID,0
symboling,0
CarName,0
fueltype,0
aspiration,0
doornumber,0
carbody,0
drivewheel,0
enginelocation,0
wheelbase,0


##2. Создадим новый признак – марку автомобиля (company). Далее исправим названия и проверим изменения.

Кардинальность признака высокая (количество категорий в категориальном признаке) - что не очень хорошо. Чтобы предсказания были более осмысленными, кардинальность надо сокращать. Потому будем рассматривать не полное название авто, а только его марку.

In [ ]:
cars.CarName.nunique()

147

In [ ]:
cars.CarName.unique()

array(['alfa-romero giulia', 'alfa-romero stelvio',
       'alfa-romero Quadrifoglio', 'audi 100 ls', 'audi 100ls',
       'audi fox', 'audi 5000', 'audi 4000', 'audi 5000s (diesel)',
       'bmw 320i', 'bmw x1', 'bmw x3', 'bmw z4', 'bmw x4', 'bmw x5',
       'chevrolet impala', 'chevrolet monte carlo', 'chevrolet vega 2300',
       'dodge rampage', 'dodge challenger se', 'dodge d200',
       'dodge monaco (sw)', 'dodge colt hardtop', 'dodge colt (sw)',
       'dodge coronet custom', 'dodge dart custom',
       'dodge coronet custom (sw)', 'honda civic', 'honda civic cvcc',
       'honda accord cvcc', 'honda accord lx', 'honda civic 1500 gl',
       'honda accord', 'honda civic 1300', 'honda prelude',
       'honda civic (auto)', 'isuzu MU-X', 'isuzu D-Max ',
       'isuzu D-Max V-Cross', 'jaguar xj', 'jaguar xf', 'jaguar xk',
       'maxda rx3', 'maxda glc deluxe', 'mazda rx2 coupe', 'mazda rx-4',
       'mazda glc deluxe', 'mazda 626', 'mazda glc', 'mazda rx-7 gs',
       'mazda glc 

In [ ]:
'audi 100 ls'.split(' ')

['audi', '100', 'ls']

In [ ]:
cars['company'] = cars.CarName.apply(lambda x : x.split(' ')[0])
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

##3. Оставим только часть предикторов, после чего посчитаем корреляцию между price и другими переменными.

Некоторые названия марок машин написаны с ошибками, переправим их и посмотрим, сколько производителей осталось в итоге

Приведем все к нижнему регистру, чтобы пофиксить случай, когда названия считаются разными, тк написаны в разном регистре или с большой и маленькой букв.

In [ ]:
cars.company = cars.company.str.lower()

Для остальных, замеченных нами случаев, напишем функцию:

In [ ]:
def fix_company(to_rep, how_rep):
    cars.company.replace(to_rep, how_rep, inplace = True)

In [ ]:
fix_company('maxda', 'mazda')
fix_company('porsche', 'porshce')
fix_company('porcshce', 'porshce')
fix_company('toyouta', 'toyota')
fix_company('vokswagen', 'volkswagen')
fix_company('vw', 'volkswagen')

In [ ]:
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porshce', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [ ]:
cars.company.nunique()

22

Отберем часть колонок

In [ ]:
cars.columns

Index(['car_ID', 'symboling', 'CarName', 'fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg',
       'price', 'company'],
      dtype='object')

In [ ]:
cols_to_keep = [ 'company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'wheelbase', 'carlength', 'carwidth',
                'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio', 'horsepower', 'price']
df = cars[cols_to_keep]
df

,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,13495.0
1,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,16500.0
2,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154,16500.0
3,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102,13950.0
4,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,four,141,3.78,114,16845.0
201,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,four,141,3.78,160,19045.0
202,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,six,173,3.58,134,21485.0
203,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,six,145,3.01,106,22470.0


In [ ]:
numeric_df = df.select_dtypes(include=['number'])
mat_cor = numeric_df.corr()

In [ ]:
mat_cor['price'].sort_values(ascending = False).round(2)

,price
price,1.00
enginesize,0.87
curbweight,0.84
horsepower,0.81
carwidth,0.76
carlength,0.68
wheelbase,0.58
boreratio,0.55


##4. Преобразуем категориальные переменные с помощью pd.get_dummies().

In [ ]:
cars.dtypes

,0
car_ID,int64
symboling,int64
CarName,object
fueltype,object
aspiration,object
doornumber,object
carbody,object
drivewheel,object
enginelocation,object
wheelbase,float64


In [ ]:
df.select_dtypes(include=['object']).columns

Index(['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel',
       'enginetype', 'cylindernumber'],
      dtype='object')

на каждую категорию создалось по отдельной переменной, чтобы не было мультиколлиниарности - ставим drop_first = True

In [ ]:
df_dummy = pd.get_dummies( data = df[['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel',
       'enginetype', 'cylindernumber']], drop_first = True)

In [ ]:
df_dummy

,company_audi,company_bmw,company_buick,company_chevrolet,company_dodge,company_honda,company_isuzu,company_jaguar,company_mazda,company_mercury,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
201,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
202,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
203,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,False


In [ ]:
cars_lr = pd.get_dummies(df, drop_first = True)
cars_lr.head()

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,88.6,168.8,64.1,2548,130,3.47,111,13495.0,False,False,...,False,False,False,False,False,True,False,False,False,False
1,88.6,168.8,64.1,2548,130,3.47,111,16500.0,False,False,...,False,False,False,False,False,True,False,False,False,False
2,94.5,171.2,65.5,2823,152,2.68,154,16500.0,False,False,...,False,False,True,False,False,False,True,False,False,False
3,99.8,176.6,66.2,2337,109,3.19,102,13950.0,True,False,...,True,False,False,False,False,True,False,False,False,False
4,99.4,176.6,66.4,2824,136,3.19,115,17450.0,True,False,...,True,False,False,False,True,False,False,False,False,False


##5. Построим модель с одним предиктором цены – horsepower.


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
results = smf.ols('price ~ horsepower', cars_lr).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Thu, 05 Sep 2024   Prob (F-statistic):           1.48e-48
Time:                        13:48:26   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

Сделаем интерсепт(значение цены, когда horsepower = 0) поянтнее, а то получается, что когда лошадиные силы = 0, нам за машину доплатят :)




In [ ]:
#центрировали, те теперь среднее новой переменной horsepower_cent = 0
cars_lr["horsepower_cent"] = cars_lr["horsepower"] - cars_lr["horsepower"].mean()

In [ ]:
results = smf.ols('price ~ horsepower_cent', cars_lr).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Thu, 05 Sep 2024   Prob (F-statistic):           1.48e-48
Time:                        13:48:26   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1.328e+04    329.446     

In [ ]:
#для машин со средним значением horsepower цена будет ~ 13 тысяч долларов
1.328e+04


13280.0

Делаем вывод по рез-м регрессии, что при увеличении лошадиных сил на единицу цена увеличивается на 163.2631 доллара(тк имеем положительный коэф-т 163 при предикторе horsepower), при уменьшении падает на 163.2631 доллара. Тк R^2 = 0.653 означает, что примерно 65.3% вариации в цене автомобилей может быть объяснено изменениями в мощности.

In [ ]:
#уберем этот, уже не нужный, столбец
cars_lr.drop("horsepower_cent", inplace = True, axis = 1)

##6. Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Выберем какую модель лучше оставить?


In [ ]:
#создадим 2 массива, в одном предикторы, в другом то, что предсказываем
X = cars_lr.drop(['price'], axis = 'columns')
Y = cars_lr['price']

#X = pd.get_dummies(X, drop_first=True)
#не забываем специально добавить интерсепт
X = sm.add_constant(X)

X

,const,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,1.0,88.6,168.8,64.1,2548,130,3.47,111,False,False,...,False,False,False,False,False,True,False,False,False,False
1,1.0,88.6,168.8,64.1,2548,130,3.47,111,False,False,...,False,False,False,False,False,True,False,False,False,False
2,1.0,94.5,171.2,65.5,2823,152,2.68,154,False,False,...,False,False,True,False,False,False,True,False,False,False
3,1.0,99.8,176.6,66.2,2337,109,3.19,102,True,False,...,True,False,False,False,False,True,False,False,False,False
4,1.0,99.4,176.6,66.4,2824,136,3.19,115,True,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,1.0,109.1,188.8,68.9,2952,141,3.78,114,False,False,...,True,False,False,False,False,True,False,False,False,False
201,1.0,109.1,188.8,68.8,3049,141,3.78,160,False,False,...,True,False,False,False,False,True,False,False,False,False
202,1.0,109.1,188.8,68.9,3012,173,3.58,134,False,False,...,False,False,True,False,False,False,True,False,False,False
203,1.0,109.1,188.8,68.9,3217,145,3.01,106,False,False,...,True,False,False,False,False,False,True,False,False,False


In [ ]:
list(enumerate(X.columns))

[(0, 'const'),
 (1, 'wheelbase'),
 (2, 'carlength'),
 (3, 'carwidth'),
 (4, 'curbweight'),
 (5, 'enginesize'),
 (6, 'boreratio'),
 (7, 'horsepower'),
 (8, 'company_audi'),
 (9, 'company_bmw'),
 (10, 'company_buick'),
 (11, 'company_chevrolet'),
 (12, 'company_dodge'),
 (13, 'company_honda'),
 (14, 'company_isuzu'),
 (15, 'company_jaguar'),
 (16, 'company_mazda'),
 (17, 'company_mercury'),
 (18, 'company_mitsubishi'),
 (19, 'company_nissan'),
 (20, 'company_peugeot'),
 (21, 'company_plymouth'),
 (22, 'company_porshce'),
 (23, 'company_renault'),
 (24, 'company_saab'),
 (25, 'company_subaru'),
 (26, 'company_toyota'),
 (27, 'company_volkswagen'),
 (28, 'company_volvo'),
 (29, 'fueltype_gas'),
 (30, 'aspiration_turbo'),
 (31, 'carbody_hardtop'),
 (32, 'carbody_hatchback'),
 (33, 'carbody_sedan'),
 (34, 'carbody_wagon'),
 (35, 'drivewheel_fwd'),
 (36, 'drivewheel_rwd'),
 (37, 'enginetype_dohcv'),
 (38, 'enginetype_l'),
 (39, 'enginetype_ohc'),
 (40, 'enginetype_ohcf'),
 (41, 'enginetype_oh

In [ ]:
X = np.asarray(X, dtype=np.float64)
#Y = np.asarray(Y, dtype=np.float64)

model_1 = sm.OLS(Y, X).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Thu, 05 Sep 2024   Prob (F-statistic):           4.86e-89
Time:                        13:48:27   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.472e+04   1.17e+04     -2.959      0.0

In [ ]:
X

array([[  1. ,  88.6, 168.8, ...,   0. ,   0. ,   0. ],
       [  1. ,  88.6, 168.8, ...,   0. ,   0. ,   0. ],
       [  1. ,  94.5, 171.2, ...,   0. ,   0. ,   0. ],
       ...,
       [  1. , 109.1, 188.8, ...,   0. ,   0. ,   0. ],
       [  1. , 109.1, 188.8, ...,   0. ,   0. ,   0. ],
       [  1. , 109.1, 188.8, ...,   0. ,   0. ,   0. ]])

Мы выкидывали какую то марку(чтобы не было идеальной мультиколлинеарности), так вот теперь ее значение как бы скрыто в интерсепте, а все остальные коэфф-ты(c x8 по х28), кт стоят при переменных, полученных из категорий марки машины(company) - это как изменится значение price если вместо той скрытой марки, возьмем данную. Похожим образом можем оценить др категориальные переменные.

Причем видим, что большинство коэф-тов, кт стоят при переменных, полученных из категорий марки машины(company), отличается от выкинутой марки не значимо!(pvalue > альфа = 0.05)

теперь проведем регрессию без company

In [ ]:
cols_to_keep = cars_lr.columns[~cars_lr.columns.str.startswith('company_')]
cols_to_keep

Index(['wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize',
       'boreratio', 'horsepower', 'price', 'fueltype_gas', 'aspiration_turbo',
       'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan',
       'carbody_wagon', 'drivewheel_fwd', 'drivewheel_rwd', 'enginetype_dohcv',
       'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf', 'enginetype_ohcv',
       'enginetype_rotor', 'cylindernumber_five', 'cylindernumber_four',
       'cylindernumber_six', 'cylindernumber_three', 'cylindernumber_twelve',
       'cylindernumber_two'],
      dtype='object')

In [ ]:
X_2 = cars_lr[cols_to_keep].drop('price', axis = 'columns')
Y_2 = cars_lr['price']

In [ ]:
X_2 = sm.add_constant(X_2)

In [ ]:
X_2 = np.asarray(X, dtype=np.float64)
#Y = np.asarray(Y, dtype=np.float64)

model_2 = sm.OLS(Y_2, X_2).fit()
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Thu, 05 Sep 2024   Prob (F-statistic):           4.86e-89
Time:                        13:48:27   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.472e+04   1.17e+04     -2.959      0.0

Видим, что у модели немного поменьше R квадрат (на 5 %  - те это явно не самый важный предиктор) - те "объяснительная сила модели" стала поменьше, но сама модель за счет уменьшения колва коэффтов стала менее перегруженной

##7. Попробуем избавиться от мультиколлинеарности

Посчитаем для каждого столбца VIF, если он меньше 5/10 - беспокоиться не о чем.
Тут же нам даже встречается значение inf у перемнной enginetype_rotor - так что переменную enginetype точно стоит модифицировать (либо сократить кол-во категорий, либо избавиться от такого предиктора в нашей моделе)

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# vif_data = pd.DataFrame()
# vif_data["feature"] = X_2.columns

# vif_data["VIF"] = [variance_inflation_factor(X_2.values, i)
#                             for i in range(len(X_2.columns))]

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

Можем использовать обобщенную линейную модель, которая не предполагает нормальность распределения отклика ни нормальность распределения ошибок


In [ ]:
model_gamma = sm.GLM(Y_2, X_2, family = sm.families.Gamma(link = sm.families.links.log())).fit()
print(model_gamma.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  price   No. Observations:                  205
Model:                            GLM   Df Residuals:                      158
Model Family:                   Gamma   Df Model:                           46
Link Function:                    log   Scale:                        0.013658
Method:                          IRLS   Log-Likelihood:                -1745.8
Date:                Thu, 05 Sep 2024   Deviance:                       2.1741
Time:                        13:51:03   Pearson chi2:                     2.16
No. Iterations:                    14   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.7097      0.749      7.620      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>